# Ray Tracing

## Grundlagen

Was ist die grundlegende Idee von Raytracing?
Wichtig ist dafür zuerst zu verstehen wie Licht funktioniert.

Ein einfacher Ansatz dafür ist, anzunehmen das Licht aus Strahlen besteht.  
Diese Strahlen werden aus einer Lichtquelle ausgesendet, werden von Objekten reflektiert und absorbiert und zu guter Letzt kommen sie im Beobachters an.  
Der Beobachter wird hier Kamera genannt.

Schematisch und zwei dimensional sieht das so aus:
![][2d_ray_tracing]

Schritt für Schritt müssen also folgende Dinge geschehen: <!--TODO bessere Formulierung-->

1. Die Lichtquelle sendet einen Lichtstrahl aus
2. Dieser Lichtstrahl trifft auf ein Objekt
3. Das Objekt reflektiert und absorbiert teil des Lichtstrahls
4. Schritt 2 und 3 wiederholen sich bis der Lichtstrahl entweder eine Kamera oder kein Objekt mehr trifft
5. Lichtstrahl trifft auf die Kamera
6. Schritt 1 bis 6 werden wiederholt bis der Nutzer mit dem gerenderten Bild zufrieden ist

<!--TODO bild durch nur kreise ersetzen-->
[2d_ray_tracing]: ./assets/2D_ray_tracing.png

### Aussenden eines Lichtstrahls

Zu Beginn wird eine einfache punktförmige Lichtquelle betrachtet, die ihre Lichtstrahlen in alle Richtungen gleich aussendet.
Die Lichtquelle kann nicht wie in der realen Welt unendlich viele Lichtstrahlen gleichzeitig aussenden.
Daher wird immer nur ein Lichtstrahl in eine zufällige Richtung von der Lichtquelle ausgesendet.
Damit ergibt sich im Durchschnitt eine gleichmäßige Aussendung von Lichtstrahlen in alle Richtungen.

Um das umsetzen zu können muss zuerst definiert werden, was ein Lichtstrahl ist.
Ein Lichtstrahl besteht aus Ortsvektor und Richtungsvektor.
Der Ortsvektor gibt dabei den Punkt an, an dem der Lichtstrahl ausgesendet wurde.  
Also $\vec{ray(t)}=\vec{O}+\vec{d}*t$

In [44]:
import numpy as np
import numpy.typing as npt
from typing import NamedTuple, Tuple

Vec2 = npt.NDArray[np.float64]

class Ray(NamedTuple):
    location: Vec2
    direction: Vec2

class LightSoruce(Vec2): # only has a location vector
    pass

def generateRayFromLightSource(lightSource: LightSoruce) -> Ray:
    # einen zufälligen winkel zwischen 0 und 2\pi generieren
    angle = np.random.rand() * 2 * np.pi

    # diesen Winkel in einen Rihtungsvektor umwandeln
    direction = np.array([np.cos(angle), np.sin(angle)])

    # daraus resultiert ein Lichtstrahl
    return Ray(lightSource, direction)

generateRayFromLightSource(np.array([1,1]))

Ray(location=array([1, 1]), direction=array([ 0.95236045, -0.30497471]))


### Schnittpunkt eines Lichtstrahls mit einem Objekt

Nachdem der Lichtstrahl ausgesendet wurde, muss der erste Auftreffpunkt des Lichtstrahls auf einem Objekt berechnet werden.
Hier werden zur Vereinfachung nur Kreise als Objekte genutzt. 
Schnittpunkte sind dabei alle Punkte, die genau den Radius $r$ des Kreises von dessen Mittelpunkt $C$ entfernt sind.
also:

$$
||\vec{P}-\vec{C}||=r
$$

Für die Lichtstrahlen gilt damit

$$
||\vec{ray(t)}-\vec{C}||=r
$$
$$
||\vec{O}+\vec{d}*t-\vec{C}|| = r
$$

umgeformt ergibt das eine Quadratische Gleichung:

$$
||\vec{d}||^2*t^2+2t*<\vec{d},\vec{O}-\vec{C}>+<\vec{O}-\vec{C},\vec{O}-\vec{C}>-r^2=0
$$

um die Schnittpunkte heruaszufinden, muss die Gleichung nacht $t$ aufgelöst werden. 
Das wird hier in code umgesetzt.

In [55]:
class Circle(NamedTuple):
    center: Vec2
    radius: float

def intersectCircle(ray :Ray, circle: Circle) -> list[float]:
    a = 1
    b = 2 * np.dot(ray.direction, ray.location - circle.center)
    c = np.linalg.norm(ray.location - circle.center) ** 2 - circle.radius ** 2
    delta = b ** 2 - 4*a*c
    if delta > 0:
        t1 = (-b + np.sqrt(delta))/(2*a)
        t2 = (-b - np.sqrt(delta))/(2*a)
        return [t1,t2]
    return []

ray = Ray(np.array([0,0]),np.array([1,0]))
circle = Circle(np.array([5,0]),1)


intersectCircle(ray, circle)

[6.0, 4.0]

Diese Berechnung muss nun für alle Objekte gemacht werden. Dabei ergeben sich voraussichtlich mehrere Werte für $t$.
Der Auftreffpunkt des Lichtstrahls ist der Punkt, für den $t$ minimal aber größer Null ist. 

### Lichtstrahl Reflexion/Absorption

### Lichtstrahl in Kamera